In [ ]:
---
description: Fill in a module description here
output-file: data.html
title: data

---



In [ ]:
#| default_exp data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio
import ccxt.async_support as ccxt
import pandas as pd
import datetime
from typing import List, Dict, Any
import numpy as np
import nest_asyncio

nest_asyncio.apply()

EXCHANGE_LIMIT_RATES: Dict[str, Dict[str, int]] = {
    "bitfinex2": {
        "limit": 10_000,
        "pause_every": 1,
        "pause": 3,
    },
    "binance": {
        "limit": 1_000,
        "pause_every": 10,
        "pause": 1,
    },
    "huobi": {
        "limit": 1_000,
        "pause_every": 10,
        "pause": 1,
    },
}


async def _ohlcv(
    exchange: ccxt.Exchange,
    symbol: str,
    timeframe: str,
    limit: int,
    step_since: int,
    timedelta: int,
) -> pd.DataFrame:
    """Asynchronously fetches OHLCV (Open, High, Low, Close, Volume) data from a cryptocurrency exchange for a specific symbol and timeframe.
    Args:
        exchange (ccxt.Exchange): The CCXT exchange object to fetch data from.
        symbol (str): The trading symbol (e.g., 'BTC/USD').
        timeframe (str): The timeframe for the OHLCV data (e.g., '1m', '1h', '1d').
        limit (int): The maximum number of OHLCV candles to fetch.
        step_since (int): The timestamp (in milliseconds) to start fetching data from.
        timedelta (int): The duration of the timeframe in milliseconds, used to calculate the closing date/time of each candle.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the OHLCV data with the following columns:
            - timestamp_open (int): The opening timestamp of the candle (in milliseconds).
            - open (float): The opening price of the candle.
            - high (float): The highest price of the candle.
            - low (float): The lowest price of the candle.
            - close (float): The closing price of the candle.
            - volume (float): The volume of the candle.
            - date_open (datetime): The opening date and time of the candle (datetime object).
            - date_close (datetime): The closing date and time of the candle (datetime object).
    """

    result = await exchange.fetch_ohlcv(
        symbol=symbol, timeframe=timeframe, limit=limit, since=step_since
    )
    result_df = pd.DataFrame(
        result, columns=["timestamp_open", "open", "high", "low", "close", "volume"]
    )
    for col in ["open", "high", "low", "close", "volume"]:
        result_df[col] = pd.to_numeric(result_df[col])
    result_df["date_open"] = pd.to_datetime(result_df["timestamp_open"], unit="ms")
    result_df["date_close"] = pd.to_datetime(
        result_df["timestamp_open"] + timedelta, unit="ms"
    )
    return result_df


async def _download_symbol(
    exchange: ccxt.Exchange,
    symbol: str,
    timeframe: str = "5m",
    since: int = int(datetime.datetime(year=2020, month=1, day=1).timestamp() * 1e3),
    until: int = int(datetime.datetime.now().timestamp() * 1e3),
    limit: int = 1000,
    pause_every: int = 10,
    pause: int = 1,
) -> pd.DataFrame:
    """Asynchronously downloads historical OHLCV data for a given symbol from a specified exchange.

    The function retrieves data in chunks, pausing periodically to avoid rate limiting.
    It then concatenates the chunks, filters them based on the specified time range,
    and performs data cleaning operations such as removing unnecessary columns,
    setting the index, sorting, dropping NaNs, and removing duplicates.

    Args:
        exchange (ccxt.Exchange): The exchange object to fetch data from.
        symbol (str): The trading symbol to download data for (e.g., 'BTC/USDT').
        timeframe (str, optional): The candlestick time frame (e.g., '5m', '1h', '1d'). Defaults to "5m".
        since (int, optional): The starting timestamp in milliseconds. Defaults to the timestamp of January 1, 2020.
        until (int, optional): The ending timestamp in milliseconds. Defaults to the current timestamp.
        limit (int, optional): The maximum number of data points to fetch per request. Defaults to 1000.
        pause_every (int, optional): The number of requests to make before pausing. Defaults to 10.
        pause (int, optional): The duration of the pause in seconds. Defaults to 1.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the historical OHLCV data, indexed by 'date_open'.
                      The DataFrame includes columns 'open', 'high', 'low', 'close', and 'volume'.
                      Returns an empty DataFrame if no data is available or if an error occurs.
    """

    # Calculate the duration of the timeframe in milliseconds
    timedelta = int(pd.Timedelta(timeframe).to_timedelta64() / 1e6)
    # Initialize lists to store tasks and results
    tasks: List[asyncio.Task] = []
    results: List[pd.DataFrame] = []
    # Loop through the time range in steps of 'limit * timedelta'
    for step_since in range(since, until, limit * timedelta):
        # Create a task to fetch OHLCV data for the current time step
        tasks.append(
            asyncio.create_task(
                _ohlcv(exchange, symbol, timeframe, limit, step_since, timedelta)
            )
        )
        # If the number of tasks reaches 'pause_every', gather the results and pause
        if len(tasks) >= pause_every:
            results.extend(await asyncio.gather(*tasks))
            await asyncio.sleep(pause)
            tasks = []
    # Gather any remaining tasks
    if len(tasks) > 0:
        results.extend(await asyncio.gather(*tasks))
    # Concatenate the results into a single DataFrame
    final_df = pd.concat(results, ignore_index=True)
    # Filter the DataFrame based on the specified time range
    final_df = final_df.loc[
        (since < final_df["timestamp_open"]) & (final_df["timestamp_open"] < until), :
    ]
    # Remove the 'timestamp_open' column
    del final_df["timestamp_open"]
    # Set the index to 'date_open'
    final_df.set_index("date_open", drop=True, inplace=True)
    # Sort the index
    final_df.sort_index(inplace=True)
    # Drop any rows with NaN values
    final_df.dropna(inplace=True)
    # Drop any duplicate rows
    final_df.drop_duplicates(inplace=True)
    return final_df


async def _download_symbols(
    exchange_name: str, symbols: List[str], dir: str, timeframe: str, **kwargs: Any
) -> None:
    """Downloads historical data for a list of symbols from a given exchange and saves it to pickle files.

    Args:
        exchange_name (str): The name of the exchange to download data from (e.g., 'binance').
        symbols (List[str]): A list of trading symbols to download data for (e.g., ['BTC/USDT', 'ETH/USDT']).
        dir (str): The directory to save the downloaded data to.
        timeframe (str): The timeframe for the data (e.g., '1d', '1h').
        **kwargs (Any): Additional keyword arguments to pass to the exchange's fetch_ohlcv method.

    Returns:
        None
    """
    exchange = getattr(ccxt, exchange_name)({"enableRateLimit": True})
    for symbol in symbols:
        df = await _download_symbol(
            exchange=exchange, symbol=symbol, timeframe=timeframe, **kwargs
        )
        save_file = f"{dir}/{exchange_name}-{symbol.replace('/', '')}-{timeframe}.pkl"
        print(f"{symbol} downloaded from {exchange_name} and stored at {save_file}")
        df.to_pickle(save_file)
    await exchange.close()


async def _download(
    exchange_names: List[str],
    symbols: List[str],
    timeframe: str,
    dir: str,
    since: datetime.datetime,
    until: datetime.datetime = datetime.datetime.now(),
) -> None:
    tasks: List[Any] = []
    for exchange_name in exchange_names:
        limit = EXCHANGE_LIMIT_RATES[exchange_name]["limit"]
        pause_every = EXCHANGE_LIMIT_RATES[exchange_name]["pause_every"]
        pause = EXCHANGE_LIMIT_RATES[exchange_name]["pause"]
        tasks.append(
            _download_symbols(
                exchange_name=exchange_name,
                symbols=symbols,
                timeframe=timeframe,
                dir=dir,
                limit=limit,
                pause_every=pause_every,
                pause=pause,
                since=int(since.timestamp() * 1e3),
                until=int(until.timestamp() * 1e3),
            )
        )
    await asyncio.gather(*tasks)


def download(*args: Any, **kwargs: Any) -> None:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(_download(*args, **kwargs))


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()